In [24]:
import pandas as pd
df = pd.read_csv("../data/sentimentdataset_cleaned.csv")
df.head()

,Id,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,...,Day,Hour,text_len,word_count,hashtag_count,emoji_count,SentimentSimple,Sentiment_norm,TextLength,LengthGroup
0,0,Enjoying a beautiful day at the park! ...,Positive,2023-01-15 12:30:00,User123,Twitter,#Nature #Park,15.0,30.0,Usa,...,15,12,52,7,2,1,Positive,Positive,52,30-59
1,1,Traffic was terrible this morning. ...,Negative,2023-01-15 08:45:00,CommuterX,Twitter,#Traffic #Morning,5.0,10.0,Canada,...,15,8,52,5,2,1,Negative,Negative,52,30-59
2,2,Just finished an amazing workout! 💪 ...,Positive,2023-01-15 15:45:00,FitnessFan,Instagram,#Fitness #Workout,20.0,40.0,Usa,...,15,15,51,6,2,2,Positive,Positive,51,30-59
3,3,Excited about the upcoming weekend getaway! ...,Positive,2023-01-15 18:20:00,AdventureX,Facebook,#Travel #Adventure,8.0,15.0,United Kingdom,...,15,18,52,6,2,1,Positive,Positive,52,30-59
4,4,Trying out a new recipe for dinner tonight. ...,Neutral,2023-01-15 19:55:00,ChefCook,Instagram,#Cooking #Food,12.0,25.0,Australia,...,15,19,52,8,2,1,Neutral,Neutral,52,30-59


debate_score=z(rt_like_ratio)+z(word_count)+z(sentiment_std)

We built a debate index for each platform using three metrics from our earlier options: the average retweet-to-like 
ratio, the average word count per post, and the standard deviation of sentiment. We standardized these metrics and added 
them up. The platform with the highest debate index is classified as the debate platform, and the other two as 
show-and-tell platforms.


Sentiment_score value counts:
sentiment_score
 1    336
-1    336
 0     60
Name: count, dtype: int64

Raw platform stats (before standardization):
           mean_rt_like_ratio  mean_word_count  sentiment_std
Platform                                                     
Facebook             0.487776        13.008658       0.956144
Instagram            0.488736        13.325581       0.958242
Twitter              0.488900        13.267490       0.963169 

Platform with highest debate_score: Twitter

Final platform classification based on debate_score:
           mean_rt_like_ratio  mean_word_count  sentiment_std  debate_score  \
Platform                                                                      
Facebook             0.487776        13.008658       0.956144     -3.827127   
Instagram            0.488736        13.325581       0.958242      1.121942   
Twitter              0.488900        13.267490       0.963169      2.705184   

          platform_type_rule  
Platform       

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report



# 建立二元標籤 (High=1, Low=0)
ret_mean = df["Retweets"].mean()
like_mean = df["Likes"].mean()
df["Retweets_bin"] = (df["Retweets"] >= ret_mean).astype(int)
df["Likes_bin"] = (df["Likes"] >= like_mean).astype(int)

# 特徵 (Text, Hashtags, SentimentSimple, Month, LengthGroup)
X = df[["Text", "Hashtags", "SentimentSimple", "Month", "LengthGroup"]]

# 前處理：文字用 TF-IDF，類別用 OneHot
preprocessor = ColumnTransformer(
    transformers=[
        ("text", TfidfVectorizer(max_features=500), "Text"),
        ("hashtags", TfidfVectorizer(max_features=200), "Hashtags"),
        ("sentiment", OneHotEncoder(handle_unknown="ignore"), ["SentimentSimple"]),
        ("month", OneHotEncoder(handle_unknown="ignore"), ["Month"]),
        ("lengthgroup", OneHotEncoder(handle_unknown="ignore"), ["LengthGroup"])
    ]
)

# Logistic Regression pipeline
log_reg = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

# 訓練 & 評估 Retweets 高低
X_train, X_test, y_train, y_test = train_test_split(X, df["Retweets_bin"], test_size=0.2, random_state=42)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print("Retweets classification report:\n", classification_report(y_test, y_pred))

# 訓練 & 評估 Likes 高低
X_train, X_test, y_train, y_test = train_test_split(X, df["Likes_bin"], test_size=0.2, random_state=42)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print("Likes classification report:\n", classification_report(y_test, y_pred))

Retweets classification report:
               precision    recall  f1-score   support

           0       0.72      0.64      0.68        67
           1       0.72      0.79      0.75        80

    accuracy                           0.72       147
   macro avg       0.72      0.71      0.72       147
weighted avg       0.72      0.72      0.72       147

Likes classification report:
               precision    recall  f1-score   support

           0       0.71      0.67      0.69        66
           1       0.74      0.78      0.76        81

    accuracy                           0.73       147
   macro avg       0.73      0.72      0.72       147
weighted avg       0.73      0.73      0.73       147



causal relationship 
if we change one parameter how will engagement be affected, draw a graph to see how the relationship changes. 
- analysis of the data, by tweaking a little bit of this variable, we can see x change in another variable. see the relationship in the variables 
- instead of predicting likes and retweets, we can do an analysis of what we're trying to find 
    relationship between sentiment and country 
    dag causal inference graph - draw something like age has affect on skin cancer, income has 
    each variable has some causal relationship to another variable 

if we want to stick to prediction models, we can use this as preprosessing step then do a model 